Import libraries

In [ ]:
!pip install requests beautifulsoup4 selenium playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 36.5 MB/s eta 0:00:00


In [ ]:
!playwright install --with-deps


Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,584 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


Crawl menu

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def crawl_pages():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Launch browser in headless mode
        page = await browser.new_page()

        # Access the list page
        url = "https://global.seoul.go.kr/web/news/sfaq/bordContListPage.do?brd_no=11"
        await page.goto(url)

        # Extract post titles and links
        posts = []
        rows = await page.query_selector_all("table tbody tr")  # Select table rows
        for row in rows:
            title = await row.query_selector("td:nth-child(2)")  # Column with title
            link = await row.query_selector("td:nth-child(2) a")  # Column with link
            if title and link:
                title_text = await title.inner_text()
                link_href = await link.get_attribute("href")
                posts.append({"title": title_text, "link": link_href})

        # Crawl each post's detail page
        for post in posts:
            post_url = f"{url}/{post['link']}"  # Convert relative path to absolute URL
            await page.goto(post_url)

            # Extract content from detail page
            content = await page.query_selector("div.content")  # Adjust selector as needed
            if content:
                post['content'] = await content.inner_text()
            else:
                post['content'] = "No content"

            print(f"title: {post['title']}\n content: {post['content']}\n")

        # Close the browser
        await browser.close()

# Run the coroutine
await crawl_pages()

title: 외국인도 국민내일배움카드를 신청할 수 있나요?
 content: No content

title: 실업급여(구직급여)를 받고 있는 외국인입니다. 새로운 직장을 빨리 찾으면 조기재취업수당을 받을 수 있나요?
 content: No content

title: 외국인도 실업급여(구직급여)를 받을 수 있나요?
 content: No content

title: 건설현장에서 일용직으로 일했습니다. 일용직도 퇴직금처럼 받는 것이 있다고 들었는데요?
 content: No content

title: 2024년 최저임금을 기준으로 기본급은 2,060,740원으로 알고 있습니다. 그런데 저의 기본급은 190만원입니다. 최저임금 위반 아닌가요?
 content: No content

title: 11개월 근무 후 체류기간연장 신청 후 허가를 기다리느라고 2주일간 일을 쉬고 다시 11개월을 일했습니다. 퇴직금을 받을 수 있나요?
 content: No content

title: 저조한 성과를 이유로 회사가 저를 해고하려고 합니다. 정당한 해고의 사유가 되나요?
 content: No content

title: 사업주의 동의를 받고 본국에 2달간 무급휴가를 다녀왔습니다. 휴가신청서 제출 시 무급휴가기간은 계속근로기간에서 제외한다고 약정했는데 유효한가요?
 content: No content

title: 개인사정으로 회사를 그만두고 싶습니다. 사직서를 제출하면 퇴사처리가 되나요?
 content: No content

title: 회사대표가 내일부터 출근하지말라고 갑자기 통보를 하였습니다. 어떻게 해야하나요?
 content: No content

title: 영주권자도 연장 신청이 필요한가요?
 content: No content

title: 체류기간 연장 신청 시 필요한 서류는 무엇인가요?
 content: No content



Extracting all content from a single webpage

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def extract_post_content():
    async with async_playwright() as p:
        # Launch browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Set the post URL
        post_url = "https://global.seoul.go.kr/web/news/sfaq/bordContDetail.do?mode=W&brd_no=11&post_no=2A72BC2BE7C701B8E063C0A8A023CF10&lang=ko"
        await page.goto(post_url)

        # Extract the post content
        try:
            # Adjust the CSS selector to match the structure of the post content
            content = await page.query_selector("div")
            if content:
                text = await content.inner_text()
                print(f"Content:\n{text}")
            else:
                print("Content not found.")
        except Exception as e:
            print(f"Error extracting content: {e}")

        # Close the browser
        await browser.close()

# Run the coroutine
await extract_post_content()


Content:
로그인
회원가입
LANGUAGE
상담정보
교육·문화
알림
서울생활안내
센터소개
알림
자주하는질문
자주하는질문
외국인도 국민내일배움카드를 신청할 수 있나요?
카테고리 : 취업 및 노무
등록자 : 총괄관리자
등록일 : 2024-12-30
조회수 : 79
2.[몽골어]2024 FAQ_10.jpg [미리보기]
1.[베트남어]2024 FAQ_10.jpg [미리보기]
[우르두어]2024 FAQ_10.jpg [미리보기]

외국인도 국민내일배움카드를 신청할 수 있나요?




□ 국민내일배움카드 발급 대상 외국인은 F-2, F-5, F-6, D-7, D-8, D-9 체류자격 소지자입니다. F-6는 고용보험 납부이력이 없어도 가능하며 나머지 체류자격은 납부이력이 있으면 발급 받을 수 있습니다. (국민내일배움카드 운영규정 제4조 제2항 제3호)

 ㅇ 국민내일배움카드를 발급받으면 5년간 300~500만원 한도 내에서 고용노동부로부터 인정받은 훈련과정을 수강하는 경우, 훈련비의 일부 또는 전액을 지원받을 수 있습니다. 가까운 고용센터를 방문하거나 고용24를 통해 신청할 수 있습니다.

목록
서울생활살피미
문화다양성 교육(교실로 찾아가는 다문화 이해교육)
사이트 링크
개인정보처리방침
이메일 무단수집거부

서울외국인주민센터 / 서울특별시 영등포구 도신로 40 (대림동 604-30) , 07379

Tel. +82-2-2229-4900 / Fax. +82-2-2229-4999 /Email. hotline@sfrc.seoul.kr

Copyright. 2003-2022 Seoul Metropolitan Government. All Rights Reserved.


Extracting text content from a single webpage

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def extract_div_content():
    async with async_playwright() as p:
        # Launch browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Set the post URL
        post_url = "https://global.seoul.go.kr/web/news/sfaq/bordContDetail.do?mode=W&brd_no=11&post_no=2A72BC2BE7C701B8E063C0A8A023CF10&lang=ko"
        await page.goto(post_url)

        try:
            # Wait for the page to load completely
            await page.wait_for_load_state("networkidle")

            # Select the <div class="content"> element
            content_div = await page.query_selector("div.content")
            if content_div:
                # Extract text
                content_text = await content_div.inner_text()
                print("\nFrequently Asked Question:")
                print(content_text)
            else:
                print("div.content element not found.")
        except Exception as e:
            print(f"Error extracting content: {e}")
        finally:
            # Close the browser
            await browser.close()

# Run the coroutine
asyncio.run(extract_div_content())



Frequently Asked Question:
외국인도 국민내일배움카드를 신청할 수 있나요?




□ 국민내일배움카드 발급 대상 외국인은 F-2, F-5, F-6, D-7, D-8, D-9 체류자격 소지자입니다. F-6는 고용보험 납부이력이 없어도 가능하며 나머지 체류자격은 납부이력이 있으면 발급 받을 수 있습니다. (국민내일배움카드 운영규정 제4조 제2항 제3호)

 ㅇ 국민내일배움카드를 발급받으면 5년간 300~500만원 한도 내에서 고용노동부로부터 인정받은 훈련과정을 수강하는 경우, 훈련비의 일부 또는 전액을 지원받을 수 있습니다. 가까운 고용센터를 방문하거나 고용24를 통해 신청할 수 있습니다.


Scraping content linked to menu tab href

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def crawl_faq_details():
    async with async_playwright() as p:
        # Launch browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Step 1: Access FAQ list page
        base_url = "https://global.seoul.go.kr"
        faq_list_url = f"{base_url}/web/news/sfaq/bordContListPage.do?brd_no=11"
        await page.goto(faq_list_url)

        # Step 2: Extract post titles and links
        faq_posts = []
        rows = await page.query_selector_all("table tbody tr")  # Select table rows
        for row in rows:
            title_element = await row.query_selector("td:nth-child(2) a")
            if title_element:
                title = await title_element.inner_text()
                href = await title_element.get_attribute("href")

                # Extract post_no from the href attribute
                if href and "contDetail" in href:
                    post_no = href.split("'")[1]  # Extract post_no from href
                    faq_posts.append({"title": title, "post_no": post_no})

        # Step 3: Crawl each post's detail page
        for faq in faq_posts:
            detail_url = f"{base_url}/web/news/sfaq/bordContDetail.do?mode=W&brd_no=11&post_no={faq['post_no']}&lang=ko"
            await page.goto(detail_url)

            # Extract content from <div class="content">
            try:
                content_div = await page.query_selector("div.content")
                if content_div:
                    faq['content'] = await content_div.inner_text()
                else:
                    faq['content'] = "No content available"
            except Exception as e:
                faq['content'] = f"Error occurred: {e}"

            # Print the extracted content
            print("\n\n")
            print(f"FAQ and Answer: {faq['content']}\n")

        # Close the browser
        await browser.close()

# Run the coroutine
asyncio.run(crawl_faq_details())





FAQ and Answer: 외국인도 국민내일배움카드를 신청할 수 있나요?




□ 국민내일배움카드 발급 대상 외국인은 F-2, F-5, F-6, D-7, D-8, D-9 체류자격 소지자입니다. F-6는 고용보험 납부이력이 없어도 가능하며 나머지 체류자격은 납부이력이 있으면 발급 받을 수 있습니다. (국민내일배움카드 운영규정 제4조 제2항 제3호)

 ㅇ 국민내일배움카드를 발급받으면 5년간 300~500만원 한도 내에서 고용노동부로부터 인정받은 훈련과정을 수강하는 경우, 훈련비의 일부 또는 전액을 지원받을 수 있습니다. 가까운 고용센터를 방문하거나 고용24를 통해 신청할 수 있습니다.




FAQ and Answer: 실업급여(구직급여)를 받고 있는 외국인입니다. 새로운 직장을 빨리 찾으면 조기재취업수당을 받을 수 있나요?




□ 조기재취업수당이란 실업급여 수급 중에 재취업일 이전 1/2이상 미지급 받은 실업급여가 있는경우, 재취업일로부터 계속근로일수가 1년이 경과한 후 신청할 수 있으며 ‘구직급여일액 x 재취업한날’을 기준으로 남아있는 소정급여일수의 1/2 금액을 받을 수 있습니다. 

다음 체류자격의 외국인만 조기재취업수당을 신청할 수 있습니다.

 ①단기취업(C-4) ②교수(E-1)~특정활동(E-7) ③계절근로(E-8) ④거주(F-2), 재외동포(F-4), 영주(F-5), 결혼이민(F-6) ⑤ 관광취업(H-1) ⑥ 외국인 선원(E-10)




FAQ and Answer: 외국인도 실업급여(구직급여)를 받을 수 있나요?




□ 실업급여는 이직전 18개월 동안 고용보험에 180일 이상 가입이 되어 있고 비자발적으로 퇴사하였을 경우에 수급대상이 됩니다

 ㅇ 외국인의 경우 체류자격에 따라 가입대상이 구분됩니다.

 당연적용(의무가입)

	

 임의가입(신청을 통해 가입 가능)

	

 적용제외




 거주(F-2)

영주(F-5) 

결혼이민(F-6)

	

 재외동포(F-4), 단기취업(C-4), 교수(E-1), 회화지도(E-2), 연구(